# DEA Intertidal Elevation

This notebook demonstrates how to run and customise a DEA Intertidal Elevation analysis.

## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [ ]:
cd ..

Install additional packages directly from the requirements file

In [ ]:
pip install -r requirements.in --quiet

### Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from ipyleaflet import basemaps, basemap_to_tiles

import datacube
import odc.geo.xr
from odc.geo.geom import Geometry
from odc.geo.geobox import GeoBox
from odc.algo import mask_cleanup, xr_quantile
from odc.ui import select_on_a_map

from dea_tools.dask import create_local_dask_cluster
from dea_tools.coastal import pixel_tides
from intertidal.io import load_data, prepare_for_export
from intertidal.elevation import (
    load_topobathy,
    ds_to_flat,
    pixel_tides_ensemble,
    pixel_rolling_median,
    pixel_dem,
    pixel_dem_debug,
    pixel_uncertainty,
    flat_to_ds,
    clean_edge_pixels,
    elevation,
)



## Setup


### Set analysis parameters

In [ ]:
# Intertidal Elevation variables
start_date = "2019"  # Start date for analysis
end_date = "2019"  # End date for analysis
resolution = 10  # Spatial resolution used for output files
crs = "EPSG:3577"  # Coordinate Reference System (CRS) to use for output files
ndwi_thresh = 0.1  # Threshold used to identify dry/wet transition
include_s2 = True  # Include Sentinel-2 data in the analysis?
include_ls = True  # Include Landsat data in the analysis?
filter_gqa = True  # Filter to remove poorly georeferenced scenes?
tide_model = "ensemble"  # Tide model to use in analysis 
# tide_model_dir = "/var/share/tide_models"  # Directory containing tide model files
# tide_model = ["FES2014", "FES2012", "TPXO9-atlas-v5"]
tide_model_dir = "/gdata1/data/tide_models_clipped"

#### Set study area

##### Option 1: load study area from 32 km tile GridSpec

In [ ]:
# Set study area (e.g. tile ID in form 'x143y87')
study_area = "x133y40"
geom = None  # Use GridSpec to load study area, not a custom geom

##### Option 2: load study area from vector file

In [ ]:
# # Set study area to load from file
# study_area = "pointstuart"

# # Read file, select study area and convert to geom
# studyarea_gdf = gpd.read_file(
#     "data/raw/intertidal_development_polygons.geojson"
# ).set_index("id")
# geom = Geometry(studyarea_gdf.loc[study_area].geometry, crs=studyarea_gdf.crs)
# geom

##### Option 3: load study area using interactive map

In [ ]:
# Set study area name for outputs
study_area = "testing"

# Plot interactive map to select area
basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
geom = select_on_a_map(height="600px", layers=(basemap,), center=(-26, 135), zoom=4)
geom

## Load data

In [ ]:
%%time

# Connect to datacube
dc = datacube.Datacube(app="Intertidal_elevation")

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

satellite_ds, _, _ = load_data(
    dc=dc,
    study_area=study_area,
    geom=geom,
    time_range=(start_date, end_date),
    resolution=resolution,
    crs=crs,
    include_s2=True,
    include_ls=True,
    filter_gqa=filter_gqa,
    max_cloudcover=90,
    skip_broken_datasets=True,
)

print(satellite_ds)

# Load data
satellite_ds.load()


In [ ]:
# # Experiment of removing mostly empty scenes to reduce memory/speed up
# satellite_ds = satellite_ds.sel(
#     time=satellite_ds.ndwi.notnull().mean(dim=["y", "x"]) > 0.9
# )

### Load optional topobathy mask
Loads a topo-bathymetric DEM for the extents of the loaded satellite data.
This is used as a coarse mask to constrain the analysis to the coastal zone, improving run time and reducing clear false positives over deep water or elevated land.

In [ ]:
# Load data from GA's Australian Bathymetry and Topography Grid 2009
topobathy_ds = load_topobathy(
    dc, satellite_ds, product="ga_multi_ausbath_0", resampling="bilinear"
)

In [ ]:
# interpolate_2d(
#         tide_lowres,
#         x_coords=ranks_gdf.geometry.x,
#         y_coords=ranks_gdf.geometry.y,
#         z_coords=ranks_gdf.point_id,
#         method=interp_method,
#     ).odc.assign_crs("EPSG:3577").astype(np.int16).odc.write_cog(f"model_nearest.tif", overwrite=True)

## Pixel-based tides

In [ ]:
# tide_model = "FES2014"
tide_model = "ensemble"

if tide_model == "ensemble":
    # Model tides into every pixel in the three-dimensional satellite
    # dataset (x by y by time) after generating ensemble model using
    # ancillary inputs
    tide_m, model_ranks = pixel_tides_ensemble(
        satellite_ds,
        directory=tide_model_dir,
        ancillary_points="data/raw/tide_correlations_2017-2019.geojson",
        top_n=3,
        reduce_method='mean',
        resolution=3000,
    )

    # Print models in order of correlation
    print(
        model_ranks.mean(dim=["x", "y"])
        .to_dataframe("ranks")
        .sort_values("ranks", ascending=False)
    )

else:
    # Model tides into every pixel in the three-dimensional satellite
    # dataset (x by y by time)
    tide_m, _ = pixel_tides(
        satellite_ds,
        resample=True,
        model=tide_model,
        directory=tide_model_dir,
    )

In [ ]:
## Experimental: testing ebb flow filtering
# ebb_flow_da, tide_m_offset = pixel_ebb_flow(tide_m, offset_min=15)

In [ ]:
# Set tide array pixels to nodata if the satellite data array pixels contain
# nodata. This ensures that we ignore any tide observations where we don't
# have matching satellite imagery
satellite_ds["tide_m"] = tide_m.where(
    ~satellite_ds.to_array().isel(variable=0).isnull().drop("variable")
)

## Pixel-based DEM creation

### Flatten array from 3D to 2D and drop pixels with no correlation with tide
Flatten array to only pixels with positive correlations between water observations and tide height. This greatly improves processing time by ensuring only a narrow strip of pixels along the coastline are analysed, rather than the entire x * y array:


![](../data/figures/tide_array_flattening.JPG)

In [ ]:
%%time
flat_ds, freq, corr = ds_to_flat(
    satellite_ds,
    ndwi_thresh=0.0,
    min_freq=0.01,
    max_freq=0.99,
    min_correlation=0.15,
    valid_mask=topobathy_ds.height_depth > -20,
)

### Pixel-wise rolling median
This function performs a rolling median calculation along the tide heights of our satellite images. 
It breaks our tide range into `windows_n` individual rolling windows, each of which covers `windows_prop_tide` of the full tidal range. 
For each window, the function returns the median of all tide heights and NDWI index values within the window, and returns an array with a new "interval" dimension that summarises these values from low to high tide.

In [ ]:
interval_ds = pixel_rolling_median(
    flat_ds,
    windows_n=100,
    window_prop_tide=0.15,
    window_offset=5,
    max_workers=None,
)

### Model intertidal elevation and uncertainty

In [ ]:
# Model elevation
flat_dem = pixel_dem(
    interval_ds,
    ndwi_thresh=ndwi_thresh,
    interp_intervals=200,
    smooth_radius=20,
)

In [ ]:
# Model uncertainty
low, high, uncertainty, misclassified = pixel_uncertainty(
    flat_ds, flat_dem, ndwi_thresh, method="mad"
)

# Add arrays to dataset
flat_dem[["elevation_low", "elevation_high", "elevation_uncertainty"]] = (
    low,
    high,
    uncertainty,
)

Plot example elevation extraction for a single pixel:

In [ ]:
x, y = 495493,-1677653
pixel_dem_debug(
    x,
    y,
    flat_ds,
    interval_ds, 
    interp_intervals=200,
    smooth_radius=20,
    plot_style="year"
)

## Unstack outputs and export

In [ ]:
# Combine QA layers with elevation layers
flat_combined = xr.combine_by_coords(
    [flat_dem,  # DEM data
     freq,  # Frequency
     corr,  # Correlation
    ],
)

# Unstack elevation and uncertainty layers back into their original 
# spatial dimensions
ds = flat_to_ds(flat_combined, satellite_ds)

# Clean upper edge of intertidal zone in elevation layers 
# (likely to be inaccurate edge pixels)
elevation_bands = [d for d in ds.data_vars if "elevation" in d]
ds[elevation_bands] = clean_edge_pixels(ds[elevation_bands])

In [ ]:
fix, axes = plt.subplots(1, 4, figsize=(12, 3))
ds.elevation.plot.imshow(cmap="viridis", ax=axes[0])
ds.elevation_uncertainty.plot.imshow(cmap="inferno", ax=axes[1])
ds.qa_ndwi_corr.plot.imshow(cmap="RdBu", vmin=-0.7, vmax=0.7, ax=axes[2])
ds.qa_ndwi_freq.plot.imshow(cmap="Blues", vmin=0, vmax=1, ax=axes[3])

### Export to GeoTIFF files

In [ ]:
# Create output folder. If it doesn't exist, create it
output_dir = f"data/interim/{study_area}/{start_date}-{end_date}"
os.makedirs(output_dir, exist_ok=True)

# Prepare for export by correctly setting nodata and dtypes, then
# export to file
prepare_for_export(ds, output_location=output_dir)

### Close Dask client

In [ ]:
client.close()